## ДЗ 4. Панков Викентий

## 1. Выровняем риды:
 ./hisat2-2.2.0/hisat2 refout -1 SRR453566_1.fastq -2 SRR453566_2.fastq > output.gff

## 2. Оценим количество экспрессирующихся генов.  
Получим статистику по количеству ридов на ген:  
 htseq-count output.sam genes.gtf  
Получим статистику по размерам генов:  
 awk -F "\t" '$3=="transcript"{print $9 $5-$4}' genes.gtf > output_length.txt
 
Расчитаем TPM:

In [72]:
import pandas as pd

lengths = pd.read_csv("genes_lengths.txt", sep = " ", index_col = False).set_index("gene")
readcounts = pd.read_csv("genes_stat.txt", sep = " ", index_col = False).set_index("gene")


In [84]:
read_length = 102
genes_number = len(readcounts)

merged = pd.merge(lengths[["gene_length"]], readcounts, left_index=True, right_index=True)
#merged = merged[merged.mapped_reads > 0]
merged["A"] = merged.mapped_reads*read_length*10**6/(merged.gene_length*genes_number)

merged["TPM"] = merged.A / merged.A.sum() * 10**6


mean_tpm = merged.TPM.mean()
merged[merged.TPM > mean_tpm]

,gene_length,mapped_reads,A,TPM
gene,,,,
YDL232W,110,144,18724.901518,493.389361
YDL229W,1841,5112,39717.947001,1046.542886
YDL228C,641,2635,58799.322945,1549.325123
YDL213C,677,262,5535.570139,145.858786
YDL212W,632,565,12787.366269,336.939047
...,...,...,...,...
YAL005C,1928,1390,10312.346711,271.723840
YAL004W,647,445,9837.969360,259.224296
YAL003W,986,2599,37703.277095,993.457603


## 3. Соберём РНК-сборку и оценим качество:  
rnaspades -1 SRR453566_1.fastq -2 SRR453566_2.fastq -o rna_assembly  
python rnaquast/rnaQUAST.py -r ref.fa -c rna_assembly/transcripts.fasta -o rna_quast_output


Avg. aligned fraction                                  0.984                    
Avg. alignment length                                  1514.783                 
Avg. mismatches per transcript                         3.19  
**95%-covered genes 3976**  
**50%-assembled isoforms 4186**